In [70]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "sqlite:///./../../app/database/database_test.db"

engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage,HumanMessage
from pydantic import Field
from typing import TypedDict
import json
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model='llama3.2:latest'
)

class ComparisonResponse(TypedDict):
    wrong_product: bool = Field(description="If Any product is wrong")
    result: str = Field(description="The result of the comparison")
    columns_table_product = [ 'id','name','description','price']
    columns_table_connectivity_specs = ['number_10_100_1000_ethernet_ports','number_of_1g_ethernet_ports_with_poe_out','number_of_1g_2_5g_5g_10g_ethernet_ports','sfp_ports','sfp_plus_ports','port_to_port_isolation','operating_system']
    columns_table_physical_specs = ['dimensions', 'diameter_x_depth','weight','packaged_weight','material', 'color', 'outdoor_rating','ip_protection','wind_load_125_mph','wind_survivability','can_be_used_indoors','can_be_used_outdoors']
    columns_table_power_specs = ['max_power_consumption','max_power_consumption_without_attachments','power_rating','current','output_power','output_voltage','input_voltage','dc_jack_input_voltage','poe_in','poe_out','poe_out_ports','poe_in_input_voltage','max_out_per_port_output_input_18_30_v','max_out_per_port_output_input_30_57_v']
    columns_table_tech_specs = ['cpu','cpu_core_count','cpu_threads_count','cpu_nominal_frequency','cpu_temperature_monitor','architecture','chipset','switch_chip_model','plc_chipset','storage_size_mb','storage_type','size_of_ram_mb','mtbf']
    columns_table_wireless_specs = ['wi_fi_generation','wireless_standards','wireless_2_4_ghz_generation','wireless_5_ghz_generation','wireless_2_4_ghz_chip_model','wireless_5_ghz_chip_model','wireless_2_4_ghz_number_of_chains','wireless_5_ghz_number_of_chains','wireless_2_4_ghz_max_data_rate','wireless_5_ghz_max_data_rate','wifi_speed','dbi','beamwidth','cross_polarization','polarization','antenna_header_count','allow_2ghz','allow_5ghz','matching','vswr']

# @tool
def product_comparator(question:str, product_id_1:str='',product_id_2:str='',product_id_3:str='',product_id_4:str=''):
    """
    Compare products and return the result according to the question provided by the user
    """
    p_columns = ["p.id","p.name","p.description","p.price"]
    c_columns = ["c.number_10_100_1000_ethernet_ports","c.number_of_1g_ethernet_ports_with_poe_out","c.number_of_1g_2_5g_5g_10g_ethernet_ports","c.sfp_ports","c.sfp_plus_ports","c.port_to_port_isolation","c.operating_system"]
    phs_columns = ["phs.dimensions","phs.diameter_x_depth","phs.weight","phs.packaged_weight","phs.material","phs.color","phs.outdoor_rating","phs.ip_protection","phs.wind_load_125_mph","phs.wind_survivability","phs.can_be_used_indoors","phs.can_be_used_outdoors"]
    ps_columns = ["ps.max_power_consumption","ps.max_power_consumption_without_attachments","ps.power_rating","ps.current","ps.output_power","ps.output_voltage","ps.input_voltage","ps.dc_jack_input_voltage","ps.poe_in","ps.poe_out","ps.poe_out_ports","ps.poe_in_input_voltage","ps.max_out_per_port_output_input_18_30_v","ps.max_out_per_port_output_input_30_57_v"]
    ts_columns = ["ts.cpu","ts.cpu_core_count","ts.cpu_threads_count","ts.cpu_nominal_frequency","ts.cpu_temperature_monitor","ts.architecture","ts.chipset","ts.switch_chip_model","ts.plc_chipset","ts.storage_size_mb","ts.storage_type","ts.size_of_ram_mb","ts.mtbf"]
    ws_columns = ["ws.wi_fi_generation","ws.wireless_standards","ws.wireless_2_4_ghz_generation","ws.wireless_5_ghz_generation","ws.wireless_2_4_ghz_chip_model","ws.wireless_5_ghz_chip_model","ws.wireless_2_4_ghz_number_of_chains","ws.wireless_5_ghz_number_of_chains","ws.wireless_2_4_ghz_max_data_rate","ws.wireless_5_ghz_max_data_rate","ws.wifi_speed","ws.dbi","ws.beamwidth","ws.cross_polarization","ws.polarization","ws.antenna_header_count","ws.allow_2ghz","ws.allow_5ghz","ws.matching","ws.vswr"]
    # select product with id join tables with product_id and product that has only id
    general_query = f"""
    SELECT { ",".join(p_columns)}, {",".join(c_columns)}, {",".join(phs_columns)}, {",".join(ps_columns)}, {",".join(ts_columns)}, {",".join(ws_columns)}
    from products p
    FULL OUTER JOIN connectivity_specs c ON p.id = c.product_id
    FULL OUTER JOIN physical_specs phs ON p.id = phs.product_id
    FULL OUTER JOIN power_specs ps ON p.id = ps.product_id
    FULL OUTER JOIN tech_specs ts ON p.id = ts.product_id
    FULL OUTER JOIN wireless_specs ws ON p.id = ws.product_id
    """
    #  validate that at least 2 products are provided
    if(product_id_1 == "" and product_id_2 == "" and product_id_3 == "" and product_id_4 == ""):
        return {"wrong_product":True, "result":"Please provide at least 2 products to compare"}
    if(product_id_1 != ""):
        general_query += f" WHERE p.id = {product_id_1}"
    if(product_id_2 != ""):
        general_query += f" OR p.id = {product_id_2}"
    if(product_id_3 != ""):
        general_query += f" OR p.id = {product_id_3}"
    if(product_id_4 != ""):
        general_query += f" OR p.id = {product_id_4}"
    
    products = engine.execute(general_query).fetchall()
    converted_products = []
    full_columns = p_columns + c_columns + phs_columns + ps_columns + ts_columns + ws_columns
    for product in products:
        product_dict = {}
        for i in range(len(full_columns)):
            value = product[i]
            if value is None or value == "":
                continue
            key = full_columns[i]
            converted_key = key.split(".")[1]
            product_dict[converted_key] = product[i]
        converted_products.append(product_dict)
    return converted_products
    

In [73]:
# product_comparator.invoke({'question': 'Which one is better?', 'product_id_1': '745', 'product_id_2': '2', 'product_id_3': '3', 'product_id_4': '4'})
product_comparator("Cual es mejor?", "745","749")

'Comparando los dos productos, puedo decirte que ambos tienen sus ventajas y desventajas. Sin embargo, voy a destacar algunas de las principales diferencias:\n\n**Hex Refresh**\n\n* Ventajas:\n + Mayor rendimiento debido al moderno ARM CPU y RAM adicional.\n + El mismo precio que su rival.\n* Desventajas:\n + Menor cantidad de puertos Ethernet (1).\n + No se mencionan características adicionales como PoE o protección contra ataques IP.\n\n**GPeRx4**\n\n* Ventajas:\n + Mayor cantidad de puertos Ethernet (4).\n + Características adicionales como PoE y protección contra ataques IP.\n* Desventajas:\n + Menor rendimiento debido a un CPU con una frecuencia más baja.\n + Un precio ligeramente menor que su rival.\n\nEn resumen, si lo que buscas es un router con mayor rendimiento y el mismo precio, el **Hex Refresh** podría ser la mejor opción. Sin embargo, si necesitas una solución más completa con características adicionales como PoE y protección contra ataques IP, el **GPeRx4** podría ser un